# Homework 1 Exercise 4
Suppose there is bias in sampling so that Land is more likely than Water to be recorded. Specifically, assume that 1-in-5 (20%) of Water samples are accidentally recorded instead as ”Land”. First, write a generative simulation of this sampling process. Assuming the true proportion of Water is 0.70, what proportion does your simulation tend to produce instead? Second, using a simulated sample of 20 tosses, compute the unbiased posterior distribution of the true proportion of water.

In [ ]:
using StatsPlots
using StatisticalRethinking: hpdi, PI
using Gen, Distributions

## First solution (only Distributions.jl)

In [ ]:
samples = 20

In [ ]:
true_water_distribution = Binomial(samples, 0.7)
true_water = rand(true_water_distribution)

In [ ]:
recorded_water_distribution = Binomial(true_water, 0.8)
recorded_water = rand(recorded_water_distribution)

## Second solution (Distributions.jl brevity)

In [ ]:
water_distribution = Binomial(samples, 0.7 * 0.8)
water = rand(water_distribution)

## Third solution (Gen.jl Bernoulli)

In [ ]:
@gen function problem4_bernoulli_model(samples=samples, water_proportion=0.7, bias=0.8)
    true_toss ~ bernoulli(water_proportion)
    no_mistake ~ bernoulli(bias)
    recorded_toss_probability = Float64(true_toss && no_mistake)
    recorded_toss ~ bernoulli(recorded_toss_probability)
    return recorded_toss
end

In [ ]:
trace = Gen.simulate(problem4_bernoulli_model, ())
Gen.get_choices(trace)

## Fourth solution (Gen.jl Binomial)

In [ ]:
@gen function problem4_binomial_model(samples=samples, water_proportion=0.7, bias=0.8)
    true_water ~ binom(samples, water_proportion)
    recorded_water ~ binom(true_water, bias)
    return recorded_water
end

In [ ]:
trace = Gen.simulate(problem4_binomial_model,())
Gen.get_choices(trace)

## Fifth solution (Gen.jl Binomial Brevity)

In [ ]:
@gen function problem4_binomial_brevity_model(samples=samples, water_proportion=0.7, bias=0.8)
    recorded_water ~ binom(samples, water_proportion * bias)
    return recorded_water
end

In [ ]:
trace = Gen.simulate(problem4_binomial_brevity_model,())
Gen.get_choices(trace)

## Calculating the true posterior

In [ ]:
tosses = 20
grid_length = 100
recorded_water = rand(Binomial(tosses, 0.7 * 0.8))
grid_p = Vector{Float64}(range(0, 1, length=grid_length))
prior = ones(grid_length)
probability_water = pdf.(Binomial.(tosses, grid_p .* 0.8), water)
posterior = probability_water .* prior
posterior_bad = pdf.(Binomial.(tosses, grid_p), water) # does not correct for bias
recorded_water

In [ ]:
plot(grid_p, [(prior ./ sum(prior)) posterior_bad posterior], title="Posteriors", xlabel="proportion water", ylabel="plausibility", labels=["prior" "posterior (wrong)" "posterior"], legend=:left)